In [ ]:
import os
import numpy as np
import pandas as pd

In [ ]:
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.pylab as pylab
import seaborn as sns

%matplotlib inline
matplotlib.style.use("ggplot")

In [ ]:
import plotly
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Read csv data

In [ ]:
data_folder = "/kaggle/input/hotel-id-2021-fgvc8/"

In [ ]:
train_df = pd.read_csv(data_folder + "train.csv", parse_dates=["timestamp"])
submission_df = pd.read_csv(data_folder + "sample_submission.csv")

# Look at csv data

## Train data

In [ ]:
train_df.head()

In [ ]:
print("Number of records: {}".format(len(train_df)))
print("Number of images: {}".format(train_df["image"].unique().size))
print("Number of chains: {}".format(train_df["chain"].unique().size))
print("Number of hotels: {}".format(train_df["hotel_id"].unique().size))
print("Newest image: {}".format(train_df["timestamp"].max()))
print("Oldest image: {}".format(train_df["timestamp"].min()))

There seems to be some duplicated images so let's look at them

In [ ]:
train_df[train_df["image"].duplicated(keep=False)]

Doesn't look that bad, just 2 duplicates but they belong to same hotel so we can simple drop them

In [ ]:
train_df = train_df.drop_duplicates(subset=["image"], keep="first")

Look at the chain values per hotel, a hotel should belong to one chain

In [ ]:
group_df = train_df.groupby("hotel_id").agg({"chain": [pd.Series.nunique, pd.Series.unique, "max"]})
group_df.columns = ["_".join(x) for x in group_df.columns.ravel()]
group_df.sort_values("chain_nunique")[::-1].head()

There are some hotels that belong to multiple chains though


In [ ]:
hotels_df = train_df[train_df["hotel_id"].isin(group_df[group_df["chain_nunique"] > 1].index)]
hotels_group_df = hotels_df.groupby(["hotel_id", "chain"]).size().to_frame("image_count").reset_index()

fig = px.bar(hotels_group_df, x="hotel_id", y="image_count", color=hotels_group_df["chain"].astype(str))
fig.update_xaxes(title_text="Hotel ID", type="category")
fig.update_yaxes(title_text="Image count")
fig.update_layout(title="Image count per hotel and chain", legend=dict(title="Chain"))
fig.show()

which is explained [here](https://www.kaggle.com/c/hotel-id-2021-fgvc8/discussion/230768#1264813) by the host:
> During the process of prepping the dataset, we discovered a few hotels that needed to be merged -- that is, there were two different IDs that referred to the same hotel. It would seem that when we did this merging, the chain ended up getting both the "unknown" label (0) and the correct label (whatever the non-zero chain number is). So you can use the non-zero label!

So we can fix the data accordingly by taking the non zero chain

In [ ]:
train_df = pd.read_csv(data_folder + "train.csv", parse_dates=["timestamp"])
group_df = train_df.groupby("hotel_id").agg({"chain": "max"}).reset_index()
train_df = train_df.merge(group_df[["hotel_id", "chain"]], on="hotel_id", suffixes=("", "_new"))

And check that the data is fixed

In [ ]:
group_df = train_df.groupby("hotel_id").agg({"chain": [pd.Series.nunique, pd.Series.unique], "chain_new": [pd.Series.nunique, pd.Series.unique]})
group_df.sort_values(("chain", "nunique"))[::-1].head()

Looks good so we can replace the old chain column with the new chain

In [ ]:
train_df["chain"] = train_df["chain_new"]
train_df.drop(columns=["chain_new"], inplace=True)

## Submission

In [ ]:
submission_df.head()

## Chains

In [ ]:
chain_group_df = train_df.groupby(["chain"]).agg({"hotel_id": [pd.Series.nunique], "image" : [pd.Series.nunique]})
chain_group_df.columns = ["_".join(x) for x in chain_group_df.columns.ravel()]
chain_group_df = chain_group_df.reset_index().sort_values("hotel_id_nunique")[::-1]

In [ ]:
# fig = make_subplots(rows=2, cols=1, vertical_spacing=0.02, shared_xaxes=True,)

# fig.add_trace(go.Bar(x=group_df["chain"].astype(str), y=group_df["hotel_id_nunique"], showlegend = False, name="Hotel count"), 1, 1)
# fig.add_trace(go.Bar(x=group_df["chain"].astype(str), y=group_df["image_nunique"], showlegend = False, name="Image count"), 2, 1)

# fig.update_yaxes(title_text="Hotel count", row=1, col=1)
# fig.update_yaxes(title_text="Image count", row=2, col=1)
# fig.update_xaxes(title_text="Hotel ID", row=2, col=1)
# fig.show()

In [ ]:
fig = px.scatter(chain_group_df, x="chain", y="hotel_id_nunique",
                 size="image_nunique", color = "image_nunique",
                 hover_name = None,
                 log_y=True, size_max=75)

fig.update_yaxes(title_text="Hotel count")
fig.update_xaxes(title_text="Chain ID")
fig.update_layout(title="Hotel and image count per chain", coloraxis=dict(colorbar=dict(title="Image count")))
fig.update_traces(hovertemplate="Chain: %{x} <br>Hotel count: %{y}<br>Image count: %{marker.size}")
fig.show()

In [ ]:
group_df = train_df.groupby(["chain", "hotel_id"]).size().to_frame("image_count").reset_index()

In [ ]:
big_chains = chain_group_df[chain_group_df["hotel_id_nunique"] >= 75]["chain"].values
mid_chains = chain_group_df[(chain_group_df["hotel_id_nunique"] < 75) & (chain_group_df["hotel_id_nunique"] >= 10)]["chain"].values
small_chains = chain_group_df[chain_group_df["hotel_id_nunique"] < 10]["chain"].values

In [ ]:
fig = px.box(group_df[group_df["chain"].isin(big_chains)], x="chain", y="image_count", height=350)
fig.update_xaxes(title_text="Chain ID", type="category")
fig.update_layout(title="Image count per hotel: Big chains (75 hotels and more)")
fig.show()

fig = px.box(group_df[group_df["chain"].isin(mid_chains)], x="chain", y="image_count", height=350)
fig.update_xaxes(title_text="Chain ID", type="category")
fig.update_layout(title="Image count per hotel: Mid chains (10-75 hotels)")
fig.show()

fig = px.box(group_df[group_df["chain"].isin(small_chains)], x="chain", y="image_count", height=350)
fig.update_xaxes(title_text="Chain ID", type="category")
fig.update_layout(title="Image count per hotel: Small chains (less than 10 hotels)")
fig.show()

## Hotels

In [ ]:
group_df = train_df.groupby(["hotel_id"]).size().to_frame("image_count").sort_values("image_count")[::-1].reset_index()

# top and low
low_df = group_df.iloc[-50:]
top_df = group_df.iloc[:50]

fig = make_subplots(rows=2, cols=2, 
                    specs=[[{"colspan": 2}, None], [{}, {}]],
                    horizontal_spacing=0.02, vertical_spacing=0.2, 
                    shared_yaxes=True,
                    subplot_titles=("", "Top 50", "Bottom 50"))


fig.add_trace(go.Scatter(x=group_df["hotel_id"], y=group_df["image_count"], showlegend = False), 1, 1)
fig.add_trace(go.Bar(x=top_df["hotel_id"], y=top_df["image_count"], showlegend = False), 2, 1)
fig.add_trace(go.Bar(x=low_df["hotel_id"], y=low_df["image_count"], showlegend = False), 2, 2)

fig.update_yaxes(title_text="Image count", row=1, col=1)
fig.update_yaxes(title_text="Image count", row=2, col=1)
fig.update_xaxes(type="category", visible=False, row=1, col=1)
fig.update_xaxes(title_text="Hotel ID", type="category", row=2, col=1)
fig.update_xaxes(title_text="Hotel ID", type="category", row=2, col=2)

fig.update_layout(title="Image count per hotel", height=550)
fig.show()

In [ ]:
fig = px.histogram(group_df, x="image_count", nbins=25, marginal="box", height=500)
fig.update_layout(title="Distribution of image count per hotel")
fig.update_traces(hovertemplate="Image count: %{x} <br>Hotel count: %{y}")
fig.show()

## Timestamps

In [ ]:
group_df = train_df.groupby([train_df["timestamp"].dt.to_period("M")])["image"].count().reset_index()

fig = px.bar(group_df, x=group_df["timestamp"].astype(str), y="image")
fig.update_yaxes(title_text="Image count")
fig.update_xaxes(title_text="Time", type="category")
fig.update_layout(title="Image count by months", height=350)
fig.show()

In [ ]:
hotel_df = train_df[train_df["hotel_id"] == 53586]
group_df = hotel_df.groupby([hotel_df["timestamp"].dt.to_period("M")])["image"].count().reset_index()

fig = px.bar(group_df, x=group_df["timestamp"].astype(str), y="image")
fig.update_yaxes(title_text="Image count")
fig.update_xaxes(title_text="Time", type="category")
fig.update_layout(title="Image count by months for hotel 53586", height=350)
fig.show()

In [ ]:
group_df = train_df.groupby([train_df["timestamp"].dt.to_period("M"), "chain"])["image"].count().reset_index()

fig = px.scatter(group_df, x=group_df["timestamp"].astype(str), y="chain",
                 size="image", color = "image",
                 hover_name = None,
                 size_max=25)

fig.update_yaxes(title_text="Chain", type="category")
fig.update_xaxes(title_text="Time")
fig.update_layout(title="Image count by months for chains", coloraxis=dict(colorbar=dict(title="Image count")))
fig.update_traces(hovertemplate="Time: %{x} <br>Chain: %{y}<br>Image count: %{marker.size}")
fig.show()

In [ ]:
chain_df = train_df[train_df["chain"] == 6]
group_df = chain_df.groupby([chain_df["timestamp"].dt.to_period("M"), "hotel_id"])["image"].count().reset_index()

fig = px.scatter(group_df, x=group_df["timestamp"].astype(str), y="hotel_id",
                 size="image", color = "image",
                 hover_name = None,
                 size_max=25)

fig.update_yaxes(title_text="Hotel", type="category")
fig.update_xaxes(title_text="Time")
fig.update_layout(title="Image count by months for hotels of chain 6", coloraxis=dict(colorbar=dict(title="Image count")))
fig.update_traces(hovertemplate="Time: %{x} <br>Hotel: %{y}<br>Image count: %{marker.size}")
fig.show()

# Look at images

In [ ]:
from PIL import Image

In [ ]:
def open_image(row_df):
    return Image.open(f"{data_folder}train_images/{row_df.chain.astype(int)}/{row_df.image}")


def select_data(data_df, chain_id, hotel_id, N):   
    if hotel_id is not None:
        sub_df = data_df[data_df["hotel_id"] == hotel_id]
    elif chain_id is not None:
        sub_df = data_df[data_df["chain"] == chain_id]
    else:
        sub_df = data_df
        
    if N is not None:
        sub_df = sub_df.sample(N)
    
    return sub_df


def show_images(data_df, nrows=None, ncols=None, fig_title=None):
    N = len(data_df)
    
    if nrows is None:
        nrows = 1
    if ncols is None:
        ncols = N
    
    fig, axs = plt.subplots(nrows, ncols, figsize=(24,int(5*nrows)))
    if not isinstance(axs, np.ndarray):
        axs = np.array(axs)
        
    axs = axs.ravel()
    
    for i in range(0, N):
        row_df = data_df.iloc[i]
        image = open_image(row_df)
        axs[i].imshow(image)
        axs[i].set_title(f"{row_df.chain}:{row_df.hotel_id}:{row_df.image}\n" + 
                          f"Time: {row_df.timestamp}\n" + 
                          f"Size: {np.shape(image)}")
        axs[i].axis("off")
        
    if fig_title is not None:
        fig.suptitle(fig_title, fontsize=16)

### Random sample of 10 images

In [ ]:
sample_df = select_data(train_df, None, None, 10)
show_images(sample_df, 2, 5)

### 5 sample images of hotel 48897

In [ ]:
sample_df = select_data(train_df, None, 48897, 5)
show_images(sample_df)

## Check image sizes

In [ ]:
N = 500

In [ ]:
sample_df = select_data(train_df, None, None, N)
x_array = []
y_array = []
z_array = []

for i in range(0, N):
    I = open_image(sample_df.iloc[i])
    x, y, z = np.shape(I)
    x_array.append(x)
    y_array.append(y)
    z_array.append(z)

In [ ]:
pd.DataFrame(data={"x": x_array, "y": y_array}).describe()

In [ ]:
fig = go.Figure()
fig.add_trace(go.Box(x=z_array, name="Z", boxpoints="all"))
fig.add_trace(go.Box(x=y_array, name="Y", boxpoints="all"))
fig.add_trace(go.Box(x=x_array, name="X", boxpoints="all"))
fig.update_yaxes(title="Axis")
fig.update_xaxes(title="Pixels")
fig.update_layout(title=f"Box plots of image dimensions based on {N} samples")
fig.show()

In [ ]:
dim_array = np.array(x_array) * y_array

fig = go.Figure()
fig.add_trace(go.Box(x=dim_array, name="X*Y", boxpoints="all"))
fig.update_xaxes(title="Pixels")
fig.update_layout(title=f"Box plots of image dimension based on {N} samples", height=250)
fig.show()

### Images with biggest dimensions

In [ ]:
max_x = sample_df.iloc[np.argmax(x_array)]
max_y = sample_df.iloc[np.argmax(y_array)]
max_dim = sample_df.iloc[np.argmax(dim_array)]

df = pd.DataFrame()
df = df.append(max_x)
df = df.append(max_y)
df = df.append(max_dim)
show_images(df)

## Chains with low image or hotel count

### Chain 18 - 1 hotel with 8 images

In [ ]:
sample_df = select_data(train_df, 18, None, None)
show_images(sample_df, 2, 4)

### Chain 58 - 2 hotels with total 13 images

In [ ]:
sample_df = select_data(train_df, 58, None, None)

#### Hotel 4300

In [ ]:
show_images(sample_df[sample_df["hotel_id"] == 4300], 2, 4)

#### Hotel 56113

In [ ]:
show_images(sample_df[sample_df["hotel_id"] == 56113], 2, 3)

## Hotels with few images

### Hotel 14964 - 1 image

In [ ]:
sample_df = select_data(train_df, None, 14964, None)
show_images(sample_df)

### Hotel 55404 - 2 images

In [ ]:
sample_df = select_data(train_df, None, 55404, None)
show_images(sample_df)

## Test images

In [ ]:
test_images = os.listdir(data_folder + "test_images/")

fig, axs= plt.subplots(1,3, figsize=(22,8))

for i in range(0, len(test_images)):
    image = Image.open(data_folder + "test_images/" + test_images[i])
    axs[i].imshow(image)
    axs[i].set_title(f"{test_images[i]}\n{np.shape(image)}")
    axs[i].axis("off")       